In [4]:
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from itertools import count
import ast

In [176]:

### Get times and clean data


def Clean_data(file, alltrials):
    with NWBHDF5IO(file, "r") as io:

        read_nwbfile = io.read()
        pre_start = read_nwbfile.acquisition["BehavioralEvents"]["presample_start_times"].timestamps[:]
        pre_stop = read_nwbfile.acquisition["BehavioralEvents"]["presample_stop_times"].timestamps[:]
        sample_start = read_nwbfile.acquisition["BehavioralEvents"]["sample_start_times"].timestamps[:]
        sample_stop = read_nwbfile.acquisition["BehavioralEvents"]["sample_stop_times"].timestamps[:]
        delay_start = read_nwbfile.acquisition["BehavioralEvents"]["delay_start_times"].timestamps[:]
        delay_stop = read_nwbfile.acquisition["BehavioralEvents"]["delay_stop_times"].timestamps[:]
        go_start = read_nwbfile.acquisition["BehavioralEvents"]["go_start_times"].timestamps[:]
        go_stop = read_nwbfile.acquisition["BehavioralEvents"]["go_stop_times"].timestamps[:]
    
    sample_start = np.intersect1d(pre_stop, sample_start)
    sample_stop = np.intersect1d(sample_stop, delay_start)
    delay_start = np.intersect1d(delay_start, sample_stop)
    delay_stop = np.intersect1d(delay_stop, go_start)
    # Add epoch's Start-Stop time pair to each trial
    alltrials_clean = alltrials.assign(pre_start = pre_start,
                             pre_stop = pre_stop,
                             sample_start = sample_start,
                             sample_stop = sample_stop,
                             delay_start = delay_start,
                             delay_stop = delay_stop,
                             go_start = go_start,
                             go_stop = go_stop)
    return alltrials_clean


def select_units(units, region):
    ### unit specification (specify region, only good units, etc)
    units['Region'] = units.electrodes.apply(lambda x: ast.literal_eval(x.location.values[0])['brain_regions']) ## adds column "Region"
    units_Data = units.query(f" Region == '{region}' ")
    units_Data = units_Data.query(" unit_quality == 'good'")
    return units_Data
def select_trials(alltrials_clean):
    #### trial specification (e.g., only hit, no early, etc)
    trials_Data = alltrials_clean.query("outcome == 'hit' ")
    trials_Data = trials_Data.query("early_lick == 'no early'")
    return trials_Data
    


In [183]:
def overview_units():
    import os 
    path = '/Users/jaramillo/map-ephys/notebook/workshop/April2024NWB/NWBdata'
    os.chdir(path)
    sub_id = []
    ses_id = []
    for dir1 in os.listdir():
        if not dir1.startswith('.'):
            print(dir[4:])
            sub_id+=[dir1]
            os.chdir((path+'/'+dir1))
            for file in (os.listdir()):
                if not file.startswith('.'):
                    print(file)
                    ses_id+=[file[15:30]]
                    ### load units and trials
                    io = NWBHDF5IO(file, "r") 
                    nwbfile = io.read()
                    units = nwbfile.units.to_dataframe()
                    units_region = units.electrodes.apply(lambda x: ast.literal_eval(x.location.values[0])['brain_regions']).value_counts()
                    print(units_region)
overview_units()   

456772
sub-455219_ses-20190805T152117_behavior+ecephys+ogen.nwb
left ALM          524
left Thalamus     505
right ALM         375
right Midbrain    296
Name: electrodes, dtype: int64
sub-455219_ses-20190807T134913_behavior+ecephys+ogen.nwb
right ALM         676
left ALM          409
right Midbrain    342
Name: electrodes, dtype: int64
sub-455219_ses-20190808T140448_behavior+ecephys+ogen.nwb
left ALM          561
left Thalamus     549
right ALM         478
right Midbrain    369
Name: electrodes, dtype: int64
sub-455219_ses-20190806T143015_behavior+ecephys+ogen.nwb
left ALM          529
right ALM         521
left Thalamus     489
right Midbrain    279
Name: electrodes, dtype: int64
456772
sub-456772_ses-20191120T130334_behavior+ecephys+ogen.nwb
left ALM          251
left Midbrain     209
right Midbrain    150
Name: electrodes, dtype: int64
sub-456772_ses-20191122T134226_behavior+ecephys+ogen.nwb
right Thalamus    364
left Thalamus     336
right ALM         322
Name: electrodes, dtype: in

In [175]:
def get_pooledandsegmented(units_Data, trials_Data):
    
#### get spikes with pooled and segmented trials between -3 and 3

    '''
    
    spikes_pooled: all spikes from units_Data are pooled (no trial distinction)
    spikes_segmented: spikes are segmented into trials, as defined by trials_Data
    
    time 0 represents the GO cue
    each trial is defined by before<time<after
    '''

    after = 1.5
    before = -2.5

    spikes_pooled = []
    spikes_segmented = []
    for unit in units_Data.reset_index()['id']:
        #print('unit', unit)
        unit_spike_times = units_Data["spike_times"][unit]
        trial_spikes = []
        spikes_pooled.append(unit_spike_times)
        for index_time, time in enumerate(trials_Data['go_start'].values):
            #print('time', time)
            # Compute spike times relative to go signal
            aligned_spikes = unit_spike_times - time
            pre_go = time - trials_Data['pre_start'].values[index_time]
            post_go = trials_Data['go_stop'].values[index_time]-time
            #print('pre_go', pre_go)
            #print('post_go', post_go)
            aligned_spikes = aligned_spikes[aligned_spikes < after]
            aligned_spikes = aligned_spikes[before < aligned_spikes]
            trial_spikes.append(aligned_spikes)
        spikes_segmented.append(trial_spikes)
    print("shape pooled spikes", np.shape(spikes_pooled))
    print("shape segmented spikes", np.shape(spikes_segmented))
    return spikes_pooled, spikes_segmented



In [202]:
def get_stats(trials_Data):
    ##### get stats to calculate selectivity
    trials_hit = trials_Data.query("outcome=='hit'").trial.values.astype("float")#np.unique((units_subject_ALM_session&{'early_lick':'no early'}&{'outcome':'hit'}).fetch('trial'))
    trials_miss = trials_Data.query("outcome=='miss'").trial.values.astype("float")#np.unique((units_subject_ALM_session&{'early_lick':'no early'}&{'outcome':'miss'}).fetch('trial'))
    trials_ignore = trials_Data.query("outcome=='ignore'").trial.values.astype("float")#np.unique((units_subject_ALM_session&{'outcome':'ignore'}).fetch('trial'))
    trials_early = trials_Data.query("early_lick=='early'").trial.values.astype("float")#np.unique((units_subject_ALM_session&{'early_lick':'early'}).fetch('trial'))
    left_hit =  trials_Data.query("trial_instruction=='left' and outcome=='hit' ").trial.values.astype("float")#np.unique((units_subject_ALM_session&{'early_lick':'no early'}&{'outcome':'hit'}&{'trial_instruction':'left'}).fetch('trial'))
    right_hit = trials_Data.query("trial_instruction=='right' and outcome=='hit' ").trial.values.astype("float")#np.unique((units_subject_ALM_session&{'early_lick':'no early'}&{'outcome':'hit'}&{'trial_instruction':'right'}).fetch('trial'))

    hit = len(trials_hit)
    miss = len(trials_miss)
    ignore = len(trials_ignore)
    early = len(trials_early)
    combinedtrials = trials_Data.trial.values.astype("float")
    '''print('hit', hit)
    print('miss',miss)
    print('ignore', ignore)
    print('early', early)
    print('percentcorrect', hit/(hit+miss))
    print('left_trials', len(left_hit))
    print('right_trials', len(right_hit))
    '''
    stats = [hit, miss, ignore, early, combinedtrials, left_hit, right_hit]
    return stats

def export_neuraldata(spikes_pooled, spikes_segmented,region, *idparams):
    import os
    import pickle
    sub_id, session_id = idparams 
    path = '/Users/jaramillo/map-ephys/notebook/workshop/April2024NWB/NWBdata/'+'sub-'+str(sub_id)+'/analysis'
    os.chdir(path)
    ###### EXPORT DATA##########
    with open(str(region)+'_notrials'+'sub-'+str(sub_id)+'_ses-'+str(session_id)+'_allunits.pkl', 'wb') as f:  # open a text file
        pickle.dump(spikes_pooled, f) # 
    with open(region+'_withtrials'+'sub-'+str(sub_id)+'_ses-'+str(session_id)+'_allunits.pkl', 'wb') as f:  # open a text file
        pickle.dump(spikes_segmented, f) # 

def export_stats(stats,*idparams):
    import os
    import pickle
    sub_id, session_id = idparams 
    path = '/Users/jaramillo/map-ephys/notebook/workshop/April2024NWB/NWBdata/'+'sub-'+str(sub_id)+'/analysis'
    os.chdir(path)
    ###### EXPORT DATA##########
    with open('session'+str(session_id)+'_sub'+str(sub_id)+'_stats.pkl', 'wb') as f: 
         pickle.dump(stats, f) # 


def get_data_allfiles(regions):
    
    '''
    Main function. It exports data related to statistics (e.g., hit, miss, etc.), spikes_pooled, spikes_segmented

    Input
    
    regions: set of regions to get the units data from
    
    '''
    import warnings
    warnings.filterwarnings('ignore')
    import os 
    path = '/Users/jaramillo/map-ephys/notebook/workshop/April2024NWB/NWBdata'
    os.chdir(path) 
    for dir1 in os.listdir():
        if not dir1.startswith('.'):
            print('dir', dir1)
            sub_id = dir1[4:]
            os.chdir((path+'/'+dir1))
            for file in (os.listdir()):
                if  file.endswith(".nwb"):
                    session_id= file[15:30]
                    datafile = file
                    print('working directory', os.getcwd())
                    print('datafile', datafile)
                    ### load NWB data
                    os.chdir((path+'/'+dir1))
                    io = NWBHDF5IO(datafile, "r") 
                    nwbfile = io.read()
                    ### load units and trials
                    units = nwbfile.units.to_dataframe()
                    alltrials = nwbfile.trials.to_dataframe()
                    alltrials_clean = Clean_data(file, alltrials)
                    trials_Data = select_trials(alltrials_clean)
                    stats = get_stats(trials_Data)
                    idparams = sub_id, session_id
                    export_stats(stats, *idparams)
                    for region in regions:
                        units_Data = select_units(units, region)
                        if units_Data.empty==False:
                            spikes_pooled, spikes_segmented = get_pooledandsegmented(units_Data, trials_Data)
                            print('session', session_id)
                            print('region',region)
                            print(type(region))
                            export_neuraldata(spikes_pooled, spikes_segmented,region, *idparams)
                            print('export complete for file ', file)
                            print()
get_data_allfiles(['left ALM', 'right ALM'])


   
    
  



dir sub-455219
working directory /Users/jaramillo/map-ephys/notebook/workshop/April2024NWB/NWBdata/sub-455219
datafile sub-455219_ses-20190805T152117_behavior+ecephys+ogen.nwb
shape pooled spikes (330,)
shape segmented spikes (330, 392)
session 20190805T152117
region left ALM
<class 'str'>
export complete for file  sub-455219_ses-20190805T152117_behavior+ecephys+ogen.nwb

shape pooled spikes (180,)
shape segmented spikes (180, 392)
session 20190805T152117
region right ALM
<class 'str'>
export complete for file  sub-455219_ses-20190805T152117_behavior+ecephys+ogen.nwb

working directory /Users/jaramillo/map-ephys/notebook/workshop/April2024NWB/NWBdata/sub-455219/analysis
datafile sub-455219_ses-20190807T134913_behavior+ecephys+ogen.nwb
shape pooled spikes (254,)
shape segmented spikes (254, 477)
session 20190807T134913
region left ALM
<class 'str'>
export complete for file  sub-455219_ses-20190807T134913_behavior+ecephys+ogen.nwb

shape pooled spikes (437,)
shape segmented spikes (437, 

In [72]:
####OLD
####EXPORT units with all trials pooled, segmented and stats

### load NWB data
io = NWBHDF5IO(file, "r") 
nwbfile = io.read()
### load units and trials
units = nwbfile.units.to_dataframe()
alltrials = nwbfile.trials.to_dataframe()

alltrials_clean = Clean_data(file)
units_Data, trials_Data = selectunitsandtrials(units, alltrials_clean)
###### EXPORT DATA##########

import pickle

with open('unitsALM_notrials'+'sub-'+str(sub_id)+'_ses-'+str(session_id)+'_allunits.pkl', 'wb') as f:  # open a text file
    pickle.dump(spikes_pooled, f) # 
with open('unitsALM_withtrials'+'sub-'+str(sub_id)+'_ses-'+str(session_id)+'_allunits.pkl', 'wb') as f:  # open a text file
    pickle.dump(spikes_segmented, f) # 
#with open('unitsThal_notrials_session'+str(session_id)+'_sub'+str(sub_id)+'_allunits.pkl', 'wb') as f:  # open a text file
    #pickle.dump(spikes_pooled_th, f) 
with open('session'+str(session_id)+'_sub'+str(sub_id)+'_stats.pkl', 'wb') as f: 
     pickle.dump(stats, f) # 
